In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from pymongo import MongoClient

In [48]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
}

def grab_data(pages=5):

    result = ''
    for page in range(1, pages):
        r = requests.get(f'https://novosibirsk.hh.ru/search/vacancy?page={page}', headers=headers)
        result+=r.content.decode()

    soup=bs(result, 'html.parser')
    vacancies = soup.find_all('div', {'class': 'vacancy-serp-item-body'})


    entries = []
    for vacancy in vacancies:

        title = (vacancy.find("a").text)
        try:
            salary = (vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'}).contents)
            currency = salary[-1]
            if 'от' in salary[0]:
                salary_min = int(salary[2].replace('\u202f',''))
                salary_max = '-'
            else:
                salary_min, salary_max = map(int, salary[0].replace('\u202f','').split(' – '));
        except:
            salary_min = salary_max = currency =  ('-')
        link = (vacancy.find('a')['href'].split('?')[0])
        entry = {'title':title,
                 'salary_min':salary_min,
                 'salary_max':salary_max,
                 'currency':currency,
                 'link':link}
        entries.append(entry)
    return entries

df = pd.DataFrame(grab_data(10), columns=['title', 'salary_min', 'salary_max', 'currency', 'link'])
df.head()

,title,salary_min,salary_max,currency,link
0,Бармен/бариста,35000,-,руб.,https://novosibirsk.hh.ru/vacancy/55308309
1,"Торговый представитель/менеджер (без поиска, п...",130000,180000,руб.,https://novosibirsk.hh.ru/vacancy/55259422
2,Администратор международного языкового учебног...,-,-,-,https://novosibirsk.hh.ru/vacancy/55277056
3,Помощник кладовщика,53000,55000,руб.,https://novosibirsk.hh.ru/vacancy/55285723
4,Мастер по производству мягкой мебели,100000,-,руб.,https://novosibirsk.hh.ru/vacancy/55264313


In [59]:
mongo = MongoClient('mongodb://localhost:27017')

db = mongo['gbru']
collection = db['headhunter']

def update_vacancies():
    added = updated = 0
    for index, row in df.iterrows():
        cur_link = row['link']
        if not collection.find_one({'link':cur_link}):
            collection.insert_one(row.to_dict())
            added+=1
        else:
            collection.update_one({'link': cur_link}, {'$set': row.to_dict()})
            updated+=1
    return {'added':added, 'updated':updated}
print(update_vacancies())

def get_salary_ht(value=50000):
    result = []
    entries = grab_data(5)
    for entry in entries:
        try:
            if value< int(entry['salary_min']) and value< int(entry['salary_max']):
                result.append(entry)
        except:
            pass
    return result
    
print(get_salary_ht())

{'added': 0, 'updated': 180}
[{'title': 'Торговый представитель/менеджер (без поиска, подтвержденные клиенты)', 'salary_min': 130000, 'salary_max': 180000, 'currency': 'руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55259422'}, {'title': 'Помощник кладовщика', 'salary_min': 53000, 'salary_max': 55000, 'currency': 'руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55285723'}, {'title': 'Комплектовщик / Упаковщик, Москва', 'salary_min': 86000, 'salary_max': 118000, 'currency': 'руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55281358'}, {'title': 'Менеджер по сопровождению клиентов РАБОТА ИЗ ДОМА', 'salary_min': 70000, 'salary_max': 135000, 'currency': 'руб.', 'link': 'https://novosibirsk.hh.ru/vacancy/55273607'}, {'title': 'Администратор', 'salary_min': 4000000, 'salary_max': 6000000, 'currency': 'сум', 'link': 'https://novosibirsk.hh.ru/vacancy/51360349'}, {'title': 'Менеджер отдела оптовых продаж', 'salary_min': 80000, 'salary_max': 120000, 'currency': 'руб.', 'link': 'https://no